In [10]:
import pandas as pd
import time # time over
import requests
import json

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from tqdm import tqdm # 반복문 진행률 표시

In [11]:
df = pd.read_csv('./test.csv', encoding = 'CP949') # pandas 라이브러리 csv 파일 읽을 때 사용
df.head(10) # df 앞에서 10개만 출력

,Unnamed: 0,error
0,240003,경북 칠곡군 왜관읍 아곡리149
1,240005,경기 이천시 증포동153-2
2,240011,경기 남양주시 화도읍 녹촌리82-5 외1필지
3,240014,"충청북도 청주시 서원구 2순환로 14901,2층"
4,240015,충남 천안시 서북구 성환읍353-13
5,240018,경북 구미시 남통동270-3
6,240019,대전광역시 동구 동서대로 1579102동 301호
7,240023,서울 서대문구 남가좌동341-1
8,240029,서울 서초구 반포동19-4 경부선터미널 10층 1호
9,240031,"경북 구미시 신평동신비로 26, 1층"


In [14]:
headers = {'user-agent':UserAgent().chrome} # 서버에서 봇으로 인지하여 차단당하는 것을 막기 위해 사용
api_key = 'API KEY 입력' # 지오코딩 API키 입력
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key=' + api_key + '&sensor=false&language=ko&address=' # python은 string 변수간에 + 연산시 하나로 합쳐짐

point_list = [] # 위도, 경도를 입력받을 list
formatted_address_list = [] # 서버에서 API를 이용하여 정형화된 주소를 입력받을 list
address1_list = [] # 광역지방자치단체(특별시, 광역시, 특별자치시, 도, 특별자치도) 입력받을 list
address2_list = [] # 기초지방자치단체(구, 군, 자치시, 군) 입력받을 list
address3_list = [] # 비자치구역(동, 읍, 면, 리 etc) 입력받을 list

api_request_count = 0 # API 요청 횟수 count

# csv 파일의 길이만큼 반복문 시행
for i in tqdm(range(len(df))): # for문 안에 tqdm 사용시 반복문 진행률 확인 가능
    address = df['error'][i] # csv 파일에서 데이터 주소데이터 입력받음
    url = base_url + str(address) # 지오코딩에서 요구하는 url 형식과 주소데이터를 함께 입력
    resp = requests.get(url, headers = headers, verify = False) # 위에서 입력받은 url을 가지고 API요청
    page = json.loads(resp.text) # API 요청 후 서버에서 제공하는 데이터를 json 타입으로 읽어온다. (크롬에 url 직접 입력하면 형식을 볼 수 있음 -> dictionary type)

    api_request_count += 1 # API를 요청했기 때문에 API 요청 횟수 증가

    if page['status'] == 'ZERO_RESULTS' : # API 요청할 때 주소값이 엉망으로 입력된 경우 'ZERO_RESULTS'값을 받는 경우
        count = -1 # count를 하나씩 감소시킴으로써 글자를 뒤에서부터 하나씩 제거함
        # 기본적인 알고리즘은 이상한 주소값이 들어온 경우 한글자씩 제거하면 가장 비슷한 주소가 입력될 것이라고 가정하여 알고리즘 작성
        # ex) 경기도 성남시 분당구 정자일로 2401층115호(정자동,월드프라자) -> 경기도 성남시 분당구 정자일로 240
        # 또한 구글 지오코딩은 띄어쓰기나 주소 오입력시 가장 비슷한 주소를 return해 주기도 함

        APPROXIMATE_FLAG = 0 # 하지만, 지오코딩이 자동으로 잡아줄 때 가끔 원하는 좌표보다 먼 곳을 잡아주는 경우를 잡기 위한 변수

        while True:
            try : # try except구문은 에러가 발생하지 않으면 try문을 실행하고, error가 발생하면 except문을 실행한다.

                address = df['error'][i][:count] # Python 텍스트 슬라이싱 방법 ex) A = '가나다라마바사'
                # A[0:5] -> A변수의 0번째 index부터 4번째 index까지 출력 -> '가나다라마' 까지만 출력된다.
                # A[:-1] -> :를 기준으로 왼쪽에 값을 입력하지 않으면 자동으로 0이 입력되고, 오른쪽엔 -1을 입력했기 때문에 뒤에서부터 첫번째 값 까지만 출력한다는 뜻
                # 즉, A[:-1] -> '가나다라마바' 까지만 출력
                
                # 위에 있는 코드와 동일
                url = base_url + address
                resp = requests.get(url, headers = headers, verify=False)
                page = json.loads(resp.text)

                api_request_count += 1

                if len(address) + count == 0 : # 글자를 뒤에서부터 하나씩 제거하다가 결국 전부 다 제거한 경우
                    if APPROXIMATE_FLAG == 1 : # 이 때 구글에서 한번이라도 비슷한 주소를 찾아준 경우 그 주소를 기반으로 값을 입력받는다.

                        # 위에 있는 코드와 동일
                        url = base_url + APPROXIMATE_formatted_address
                        resp = requests.get(url, headers = headers, verify=False)
                        page = json.loads(resp.text)

                        api_request_count += 1

                        # 구글에 API 요청 후 제공하는 데이터가 딕셔너리기 때문에 아래와 같이 위도와 경도값을 입력받는다.
                        lat = str(page['results'][0]['geometry']['location']['lat'])
                        lng = str(page['results'][0]['geometry']['location']['lng'])

                        # 주소를 입력하는 값에 따라 도, 시, 구 등을 return 해주는 경우도 있고 아닌 경우도 있기 때문에 배열 index 접근이 아닌 조건문을 써서 접근한다.
                        for j in page['results'][0]['address_components'] :
                            # administrative_area_level_1 은 도, 특별시와 같은 행정구역을 return
                            if 'administrative_area_level_1' in j['types'] :
                                address1 += j['long_name']
                            # localtiy는 시, 군과 같은 행정구역을 return
                            elif 'locality' in j['types'] :
                                address2_1 += j['long_name']
                            # sublocality_level_1은 구와 같은 행정구역을 return 
                            elif 'sublocality_level_1' in j['types'] :
                                address2_2 += j['long_name']
                            # sublocality_level_2는 동과 같은 행정구역을 return
                            elif 'sublocality_level_2' in j['types'] :
                                address3_1 += j['long_name']
                            # sublocality_level_4는 도로와 같은 행정구역을 return  ex)화중로
                            elif 'sublocality_level_4' in j['types'] :
                                address3_2 += j['long_name']

                        # 경도와 위도를 넣을 때 중간에 ',' 문자를 넣어서 경도와 위도를 구분지어줌
                        temp_str = lng + ',' + lat
                        point_list.append(temp_str)

                        # 구글에서 비슷한 주소를 찾아주는 경우 앞에 대한민국이라는 주소와 뒤에 KR이라는 불필요한 값을 함께 주는 경우가 있음
                        # 뒤에 KR이 있는 경우 -3까지 슬라이싱을 하여 KR제거
                        if 'KR' in APPROXIMATE_formatted_address :
                            formatted_address_list.append(APPROXIMATE_formatted_address[5:-3])
                        # 뒤에 KR이 없는 경우에는 대한민국만 제거
                        else :
                            formatted_address_list.append(APPROXIMATE_formatted_address[5:])

                        address1_list.append(address1)

                        # 시와 구는 함께 넣어야 하기 때문에 아래와 같이 진행
                        # 시가 존재하지 않는 경우에는 구만 입력
                        if len(address2_1) == 0 :
                            address2_list.append(address2_2)
                        # 시가 존재하는 경우에는 시와 구를 함께 입력 ex) 고양시 덕양구
                        else :
                            address2_list.append(address2_1 + ' ' + address2_2)
                        
                        # 구글 지오코딩의 경우 도로명 주소를 넣으면 도로명 주소값으로 주고 지번주소를 넣으면 지번주소를 return하기 때문에
                        # 동 주소가 존재하는 경우에는 동 주소로 넣고 아니면 도로명 주소로 넣는다.
                        if len(address3_1) == 0 :
                            address3_list.append(address3_2)
                        else :
                            address3_list.append(address3_1)
                        break
                    
                    # 모든 경우를 만족하지 않는 경우에는 ERROR가 발생했다고 처리하고 다른 코드와 cross check를 통해 해결
                    # 이마저도 안되면 직접 수기로 해결한다.
                    else : 
                        point_list.append('ERROR OCCURED.')
                        formatted_address_list.append('ERROR OCCURED.')
                        address1_list.append("ERROR OCCURED.")
                        address2_list.append("ERROR OCCURED.")
                        address3_list.append("ERROR OCCURED.")
                        break
                
                elif page['status'] == 'ZERO_RESULTS' :
                    count -= 1
                    continue
                
                # 구글 지오코딩의 경우에는 'location_type'이 4가지가 존재한다. 자세한 사항은 https://developers.google.com/maps/documentation/geocoding/overview 에 설명되어있다.
                # 'ROOFTOP' -> indicates that the returned result is a precise geocode for which we have location information accurate down to street address precision.
                # 'RANGE_INTERPOLATED' -> indicates that the returned result reflects an approximation (usually on a road) interpolated between two precise points (such as intersections). 
                #                         Interpolated results are generally returned when rooftop geocodes are unavailable for a street address.
                # 'GEOMETRIC_CENTER' -> indicates that the returned result is the geometric center of a result such as a polyline (for example, a street) or polygon (region).
                # 'APPROXIMATE' -> indicates that the returned result is approximate.
                
                # 가장 정확한 값을 return하는 ROOFTOP을 최대한 사용하기 위해 코드를 구성, 뒤에서 한글자씩 제거를 함에도 ROOFTOP을 사용하지 못한 경우에는 APPROXIMATE 값을 이용하는 코드로 구성
                
                # 구글 지오코딩에서 한번이라도 'APPROXIMATE'값을 return해준 경우 조건을 check하기 위해 APPROXIMATE_FLAG 변수를 1로 바꿔준다.
                elif page['results'][0]['geometry']['location_type'] == 'APPROXIMATE' :
                    if APPROXIMATE_FLAG == 0 : # APPROXIMATE를 한번이라도 return해준 경우 APPROXIMATE일 때 정제된 주소를 변수에 넣어 기억해준다.
                        APPROXIMATE_formatted_address = page['results'][0]['formatted_address']
                        APPROXIMATE_FLAG = 1 # 단 한번이라도 APPROXIMATE값을 입력받은 경우를 check하기 위한 변수
                    else : # APPROXIMATE 값을 받았는데, 이미 APPROXIMATE_FLAG가 1인 경우에는 글자를 한글자씩 감소시켜 ROOFTOP이 될 때 까지 API를 요청해본다.
                        count -= 1
                    continue
                
                # 'ROOFTOP'은 가장 정확한 주소를 입력한 경우이기 때문에 모든 값을 변수에 저장한다.
                elif page['results'][0]['geometry']['location_type'] == 'ROOFTOP' :

                    # 위에 있는 코드와 동일
                    lat = str(page['results'][0]['geometry']['location']['lat'])
                    lng = str(page['results'][0]['geometry']['location']['lng'])
                    formatted_address = page['results'][0]['formatted_address']

                    address1 = ''
                    address2_1 = ''
                    address2_2 = ''
                    address3_1 = ''
                    address3_2 = ''

                    for j in page['results'][0]['address_components'] :
                        if 'administrative_area_level_1' in j['types'] :
                            address1 += j['long_name']
                        elif 'locality' in j['types'] :
                            address2_1 += j['long_name']
                        elif 'sublocality_level_1' in j['types'] :
                            address2_2 += j['long_name']
                        elif 'sublocality_level_2' in j['types'] :
                            address3_1 += j['long_name']
                        elif 'sublocality_level_4' in j['types'] :
                            address3_2 += j['long_name']

                    temp_str = lng + ',' + lat
                    point_list.append(temp_str)

                    if 'KR' in formatted_address :
                        formatted_address_list.append(formatted_address[5:-3])
                    else :
                        formatted_address_list.append(formatted_address[5:])

                    address1_list.append(address1)

                    if len(address2_1) == 0 :
                        address2_list.append(address2_1 + address2_2)
                    else :
                        address2_list.append(address2_1 + ' ' + address2_2)
                    
                    if len(address3_1) == 0 :
                        address3_list.append(address3_2)
                    else :
                        address3_list.append(address3_1)
                    break

                else :
                    count -= 1
            except :
                time.sleep(5)
                continue
                
    else: # 처음 시도부터 정확한 주소값을 받은 경우

        lat = str(page['results'][0]['geometry']['location']['lat'])
        lng = str(page['results'][0]['geometry']['location']['lng'])
        formatted_address = page['results'][0]['formatted_address']

        address1 = ''
        address2_1 = ''
        address2_2 = ''
        address3_1 = ''
        address3_2 = ''

        for j in page['results'][0]['address_components'] :
            if 'administrative_area_level_1' in j['types'] :
                address1 += j['long_name']
            elif 'locality' in j['types'] :
                address2_1 += j['long_name']
            elif 'sublocality_level_1' in j['types'] :
                address2_2 += j['long_name']
            elif 'sublocality_level_2' in j['types'] :
                address3_1 += j['long_name']
            elif 'sublocality_level_4' in j['types'] :
                address3_2 += j['long_name']

        temp_str = lng + ',' + lat
        point_list.append(temp_str)

        if 'KR' in formatted_address :
            formatted_address_list.append(formatted_address[5:-3])
        else :
            formatted_address_list.append(formatted_address[5:])

        address1_list.append(address1)

        if len(address2_1) == 0 :
            address2_list.append(address2_1 + address2_2)
        else :
            address2_list.append(address2_1 + ' ' + address2_2)

        if len(address3_1) == 0 :
            address3_list.append(address3_2)
        else :
            address3_list.append(address3_1)

100%|██████████| 50/50 [02:08<00:00,  2.58s/it]


In [15]:
# csv 파일의 갯수와 API를 요청하여 return 받은 값의 갯수가 같은지 확인
result_count = len(point_list)
result_count

50

In [16]:
# API 요청갯수 확인
api_request_count

71

In [27]:
# input 값으로 넣어준 csv 파일의 갯수와 API를 요청하여 return 받은 값의 갯수가 다른 경우에만 사용
# pandas dataframe에 row 갯수를 맞추지 않고 대입할 경우 error가 발생하기 때문에 슬라이싱을 이용하여 처리

# df = df.drop(df.index[result_count:])
# df['point'] = point_list[:result_count]
# df['formatted_address'] = formatted_address_list[:result_count]
# df['address1'] = address1_list[:result_count]
# df['address2'] = address2_list[:result_count]
# df['address3'] = address3_list[:result_count]

In [17]:
# input 값으로 넣어준 csv 파일의 갯수와 API를 요청하여 return 받은 값의 갯수가 같은 경우

df['point'] = point_list
df['formatted_address'] = formatted_address_list
df['address1'] = address1_list
df['address2'] = address2_list
df['address3'] = address3_list

In [18]:
df.head(50)

,Unnamed: 0,error,point,formatted_address,address1,address2,address3
0,240003,경북 칠곡군 왜관읍 아곡리149,"128.4301263,36.0140985",경상북도 칠곡군 왜관읍 아곡리 149,경상북도,칠곡군,왜관읍
1,240005,경기 이천시 증포동153-2,"127.454034,37.2898498",경기도 이천시 증포동 153-2,경기도,이천시,증포동
2,240011,경기 남양주시 화도읍 녹촌리82-5 외1필지,"127.2979878,37.6446446",경기도 남양주시 화도읍 녹촌리 82-5,경기도,남양주시,화도읍
3,240014,"충청북도 청주시 서원구 2순환로 14901,2층","127.4381966,36.6333448",충청북도 청주시 흥덕구 2순환로,충청북도,청주시 흥덕구,2순환로
4,240015,충남 천안시 서북구 성환읍353-13,"127.131046,36.9166037",충청남도 천안시 서북구 성환읍 성환리 353-13,충청남도,천안시 서북구,성환읍
5,240018,경북 구미시 남통동270-3,"128.3151662,36.1116661",경상북도 구미시 남통동 270-3,경상북도,구미시,남통동
6,240019,대전광역시 동구 동서대로 1579102동 301호,"127.4301253,36.3454714",대전광역시 동구 동서대로,대전광역시,동구,동서대로
7,240023,서울 서대문구 남가좌동341-1,"126.9233067,37.5780482",서울특별시 서대문구 남가좌동 341-1,서울특별시,서대문구,남가좌동
8,240029,서울 서초구 반포동19-4 경부선터미널 10층 1호,"127.006719,37.5054942",서울특별시 서초구 반포동 19-4,서울특별시,서초구,반포동
9,240031,"경북 구미시 신평동신비로 26, 1층","128.3652642,36.1221708",경상북도 구미시 신평2동 신비로 26 1층,경상북도,구미시,신평2동


In [20]:
# 처리한 데이터를 csv파일로 다시 저장하는 작업
df.to_csv('formatted_result.csv', encoding = 'euc-kr', index = False)